# DataLoader example

This is a copy of the neural network classifier 1 with some minor changes, so if you haven't gone through that notebook, then do that first!

Changes:
- DataSet class to handle the data
- DataLoader to support batching and shuffling

In [1]:
from collections import Counter
import random

import torch
from torch.utils.data import Dataset, DataLoader

from tokenizers import Tokenizer
from tqdm.notebook import tqdm
import evaluate

import torch.nn.functional as F  # shorthand so we can do F.softmax and other functions

In [2]:
def read_data():
    tokenizer = Tokenizer.from_pretrained("bert-base-cased")

    train = []
    with open("SH-TTC/train.tsv") as fin:
        for line in fin:
            label, text = line.strip().split("\t")
            tokens = tokenizer.encode(text).tokens
            train.append((label, tokens))

    dev = []
    with open("SH-TTC/dev.tsv") as fin:
        for line in fin:
            label, text = line.strip().split("\t")
            tokens = tokenizer.encode(text).tokens
            dev.append((label, tokens))
    
    return train, dev

train_data_raw, dev_data_raw = read_data()

In [3]:
class Vocab:
    def __init__(self, tokens):
        self.vocab = [tok for tok, count in Counter(tokens).most_common()]
        self.tok2idx = {tok: idx + 1 for idx, tok in enumerate(self.vocab)}
        self.tok2idx[0] = "[UNK]"
        self.idx2tok = {idx: tok for tok, idx in self.tok2idx.items()}
    
    def __len__(self):
        return len(self.tok2idx)
    
    def to_id(self, tok):
        return self.tok2idx.get(tok, 0)

    def to_tok(self, id):
        return self.idx2tok.get(id, "[UNK]")

In [4]:
vocab = Vocab([word
               for y, x in train_data_raw
               for word in x])

In [ ]:
len(vocab)

In [6]:
class ShttcDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = []
        for y, x in tokenized_data:
            tokens = x
            x = torch.zeros(len(vocab))
            for tok in tokens:
                x[vocab.to_id(tok)] += 1
            
            if y == "SH":
                y = torch.Tensor([1, 0])
            else:
                y = torch.Tensor([0, 1])
                
            self.data.append((x, y))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [7]:
train_data = ShttcDataset(train_data_raw)
dev_data = ShttcDataset(dev_data_raw)

In [8]:
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size=8)  # not training, so no need to shuffle

In [ ]:
len(train_data), len(dev_data)

In [ ]:
len(train_loader), len(dev_loader)

In [ ]:
train_data[0]

In [ ]:
# notice the batch dimension is added on as the first dimension!
for x, y in train_loader:
    print(x)
    print(x.shape)
    print(y)
    print(y.shape)
    break

## Model!

We are using the same model as NN clasifier 1 (just a single linear layer into 2 outputs).

In [13]:
# step through the model
lin = torch.nn.Linear(len(vocab), 2)

In [14]:
yhat = lin(x)  # works with a batch!

In [ ]:
yhat

In [ ]:
yhat.shape

In [ ]:
F.cross_entropy(yhat, y)  # this averages the loss for the whole batch

In [18]:
class NNClassifier(torch.nn.Module):
    def __init__(self, voc_size):
        super().__init__()
        self.linear = torch.nn.Linear(voc_size, 2)  # Wx + b

    def forward(self, x):
        y = self.linear(x)
        return y  # don't need softmax because cross_entropy loss automatically computes it

In [19]:
model = NNClassifier(len(vocab))

In [ ]:
def count_parameters(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        print(name, "\t", params)
        total_params += params
    print(f"Total Trainable Params: {total_params}")
    
    
count_parameters(model)

In [ ]:
# get prediction for a single data point
# no_grad means we don't need to calculate gradients
# (do this when testing the model)
with torch.no_grad():
    x = train_data[0][0]
    print(model(x))

In [22]:
# setup the training
loss_func = F.cross_entropy  # same as torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# train!
for epoch in range(10):
    print("Epoch", epoch)
    
    for x, y in train_loader:  # use train_loader instead! Automatic batching and shuffle
        model.zero_grad()  # do this before running

        pred = model(x)
        loss = loss_func(pred, y)
        loss.backward()  # calculate gradients
        optimizer.step()  # updates thetas

    # after each epoch, check how we're doing
    # compute avg loss over train and dev sets
    with torch.no_grad():
        total_loss = 0
        for x, y in train_loader:
            pred = model(x)
            loss = loss_func(pred, y)
            total_loss += loss
        print("train loss:", total_loss / len(train_loader))

        total_loss = 0
        for x, y in dev_loader:
            pred = model(x)
            loss = loss_func(pred, y)
            total_loss += loss
        print("dev loss:", total_loss / len(dev_loader))

Notice that it is much faster to get through one epoch. Yay!

In [24]:
def run_model_on_dev_data():
    preds = []
    with torch.no_grad():
        for x, y in dev_data:
            pred = model(x)  # pred is something like [0.6, 0.4]
            preds.append(pred)
    return preds

def sample_predictions(preds):
    for _ in range(5):
        idx = random.randint(0, len(dev_data))
        
        # argmax gives the index with the highest value
        pred_label = "SH" if torch.argmax(preds[idx]) == 0 else "TTC"

        print("Input:", " ".join(dev_data_raw[idx][1]))
        print("Gold: ", dev_data_raw[idx][0])

        # preds are not normalized, so for better viewing, run it through softmax
        print("Pred: ", pred_label, F.softmax(preds[idx], dim=0)) 
        print()

In [ ]:
preds = run_model_on_dev_data()
sample_predictions(preds)

In [26]:
precision = evaluate.load("precision")
recall = evaluate.load("recall")
accuracy = evaluate.load("accuracy")

In [ ]:
# evaluate functions require numeric data, so convert labels to 0 and 1
refs = []
for label, text in dev_data_raw:
    if label == "SH":
        refs.append(0)
    else:
        refs.append(1)

preds_binary = []
for pred in preds:
    preds_binary.append(torch.argmax(pred))

print(precision.compute(references=refs, predictions=preds_binary))
print(recall.compute(references=refs, predictions=preds_binary))
print(accuracy.compute(references=refs, predictions=preds_binary))

## Your Tasks

Change the batch size to see how it affects training.

Then go through the NN classifier 2 notebook (with embeddings) and add a dataloader 